In [1]:
import numpy as np

In [2]:
### Hepler function ###
def fill_area(start_row, start_col, target, recipe, multiplier):
    """ Hepler function for filling out subarea of target matrix 
        with content of recipe * multiplier
    Args:
        start_row : Integer = The beginning row index of area
        start_col : Integer = The beginning col index of area
        target    : 2D numpy array to be filled in subarea
        recipe    : 2D numpy array that goes into subarea of target
        multiplier: Float (possibly complex) multiplied onto all recipe vals
    """
    row_counter = 0
    for row in range(start_row,start_row + 2):
        col_counter = 0
        for col in range(start_col, start_col + 2):
            target[row][col] = recipe[row_counter][col_counter] * multiplier
            col_counter += 1
        row_counter+=1

In [3]:
### Main functions ###


def tensor_state(str):
    """ Creates a numpy vector representation of a tensor
    product state consisting of |0> 's and |1> 's

    Args:
        string of zeros and ones, e.g. '10'

    Returns:
        np.array([0,1,0,0,...])
    """
    zero, one = np.array([1,0]), np.array([0,1])
    basis_objects = [zero,one]
    objects = []
    objects.append(basis_objects[int(str[-1])])

    for idx in range(0 , len(str) - 1):
        current_object = np.zeros(2**(idx+2))
        counter = 0
        for i in range(2):
            for j in range(len(objects[-1])):
                current_object[counter] = objects[-1][j] * basis_objects[int(str[idx])][i]
                counter += 1
        objects.append(current_object)
    return objects[-1]


def multiple_matrix_tensor(string):
    """ Creates series of tensor product of square matrix repr.
    of multiple pauli operators 

    Args:
        string: A string representation of the pauli operators
                where X = Pauli_X, Y = Pauli_Y, Z = Pauli_Z, I = identity
    
    Returns:
        2D numpy array corresponding to matrix repr. of tensor product
    """
    complex_vals = False
    for str in string:
        if str == "Y": complex_vals = True

    sigma_x, sigma_y, sigma_z = np.array([[0,1],[1,0]]),np.array([[0,-1j],[1j,0]]),np.array([[1,0],[0,-1]])
    identity = np.identity(2)
    basis_map = {"X":sigma_x,"Y":sigma_y,"Z":sigma_z,"I":identity}

    objects = []
    objects.append(basis_map[string[0]])
    for idx in range(0 , len(string) - 1):
        if complex_vals:
            current_object = np.zeros((2 ** (idx+2) , 2 ** (idx+2)),dtype=complex)
        else:
            current_object = np.zeros((2 ** (idx+2) , 2 ** (idx+2)))
        row_counter = 0
        for i in range(int(current_object.shape[0] / 2)):
            col_counter = 0
            for j in range(int(current_object.shape[0] / 2)):
                if objects[-1][i][j] == 1:
                    fill_area(row_counter,col_counter,current_object,basis_map[string[idx+1]],1)
                elif objects[-1][i][j] == -1:
                    fill_area(row_counter,col_counter,current_object,basis_map[string[idx+1]],-1)
                elif objects[-1][i][j] == 1j:
                    fill_area(row_counter,col_counter,current_object,basis_map[string[idx+1]],1j)
                elif objects[-1][i][j] == -1j:
                    fill_area(row_counter,col_counter,current_object,basis_map[string[idx+1]],-1j)
                col_counter += 2
            row_counter += 2                
        objects.append(current_object)
    return objects[-1]


def initial_MSA_matrix(strings):
    """ Creating a matrix representation of the strings given
    and filling gaps with "_"

    Args:
        list of strings, e.g. ["ACCT","AC","AT"]

    Returns:
        2D numpy array
    """
    lengths = [len(str) for str in strings]
    initial_matrix = np.zeros((len(strings) , np.max(lengths)),dtype=object)
    for row in range(initial_matrix.shape[0]):
        for col in range(len(strings[row])):
            initial_matrix[row][col] = strings[row][col]
    initial_matrix[initial_matrix == 0] = "_"
    return initial_matrix

def MSA_binary_matrix(matrix):
    """ Mapping matrix of alphabet strings to 
    corresponding binary strings 
    
    Args:
        2D numpy array, e.g. array([['A', 'C', 'C', 'T'],
                                    ['A', 'C', '_', '_'],
                                    ['A', 'T', '_', '_']])
    Returns:
        Corresponding mapped 2D numpy array
    """
    alphabet_dict = {"_":'000',"A":'001',"C":'010',"T":'011',"G":'100'}
    resulting_matrix = np.zeros(matrix.shape,dtype=object)
    for row in range(matrix.shape[0]):
        for col in range(matrix.shape[1]):
            binary = alphabet_dict[matrix[row][col]]
            resulting_matrix[row][col] = binary
    return resulting_matrix

def hamming_distance(state1,state2):
    """Calculates the hamming distance between 
    state1 and state2 

    Args:
        state1: string repr. of tensor state, e.g. "001" or "111" or ...
        state2: string repr. of tensor state, e.g. "001" or "111" or ...
    Returns:
        dist: integer score equivalent to number of differing characters
    """
    dist = 0
    for i in range(0 , len(state1)):
        if state1[i] != state2[i]: dist += 1
    return dist 

In [4]:
initial_strings = ["ACCT","AC","AT"]
my_mat = initial_MSA_matrix(initial_strings)
my_mat

array([['A', 'C', 'C', 'T'],
       ['A', 'C', '_', '_'],
       ['A', 'T', '_', '_']], dtype=object)

In [5]:
binary_repr = MSA_binary_matrix(my_mat)
binary_repr

array([['001', '010', '010', '011'],
       ['001', '010', '000', '000'],
       ['001', '011', '000', '000']], dtype=object)

In [6]:
some_state = binary_repr[0][0]
some_state

'001'

In [7]:
tensor_state(some_state)

array([0., 1., 0., 0., 0., 0., 0., 0.])

In [8]:
multiple_matrix_tensor("Z")

array([[ 1,  0],
       [ 0, -1]])

In [9]:
test = np.array([1,2,3,4,5])
for i in range(0,len(test) - 1):
    for j in range(i+1,len(test)):
        print(test[i],test[j])

1 2
1 3
1 4
1 5
2 3
2 4
2 5
3 4
3 5
4 5


In [11]:
def projection_operator(state):
    """Creates a projection operator in the basis of given state
    
        Args:
            State: numpy array 

        Returns:
            2D numpy array 
    """

    return np.outer(state,state)
    